# 

In [1]:
!pip install pandas numpy==1.26.4 matplotlib pyarrow scikit-learn tkan kamoe "jax[cuda12]"

  Attempting uninstall: kamoe
    Found existing installation: kamoe 0.1.7
    Uninstalling kamoe-0.1.7:
      Successfully uninstalled kamoe-0.1.7


In [2]:
import os
BACKEND = 'jax' # You can use any backend here 
os.environ['KERAS_BACKEND'] = BACKEND

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Flatten, GRU

from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error

from tkan import TKAN
from kamoe import MoE, KAMoE

import time

keras.utils.set_random_seed(1) 

N_MAX_EPOCHS = 1000
BATCH_SIZE = 128
early_stopping_callback = lambda : keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=10,
    mode="min",
    restore_best_weights=True,
    start_from_epoch=6,
)
lr_callback = lambda : keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=5,
    mode="min",
    min_delta=0.00001,
    min_lr=0.000025,
    verbose=0,
)
callbacks = lambda : [early_stopping_callback(), lr_callback(), keras.callbacks.TerminateOnNaN()]


# Data

In [3]:
df = pd.read_parquet('/workspace/data.parquet')
df = df[(df.index >= pd.Timestamp('2020-01-01')) & (df.index < pd.Timestamp('2023-01-01'))]
assets = ['BTC', 'ETH', 'ETC', 'XRP', 'BCH', 'LTC']
df = df[[c for c in df.columns if 'quote asset volume' in c and any(asset in c for asset in assets)]]
df.columns = [c.replace(' quote asset volume', '') for c in df.columns]
display(df)

,BTC,LTC,ETC,ETH,XRP,BCH
group,,,,,,
2020-01-01 00:00:00,3.675857e+06,1.165827e+05,56488.402352,1.000930e+06,2.579254e+05,178258.749391
2020-01-01 01:00:00,6.365953e+06,2.830715e+05,182102.074213,1.474278e+06,4.520609e+05,615321.025242
2020-01-01 02:00:00,4.736719e+06,2.402871e+05,134063.422732,9.940256e+05,4.414948e+05,221535.645771
2020-01-01 03:00:00,5.667367e+06,1.613043e+05,131094.172168,6.473610e+05,1.886061e+05,397185.950571
2020-01-01 04:00:00,3.379094e+06,2.214516e+05,134937.122201,4.430067e+05,2.279373e+05,316499.137509
...,...,...,...,...,...,...
2022-12-31 19:00:00,6.704605e+07,9.964355e+05,181340.756100,7.738029e+06,1.413563e+06,35409.149500
2022-12-31 20:00:00,4.344849e+07,1.173164e+06,90513.087600,4.278879e+06,1.113527e+06,42674.516600
2022-12-31 21:00:00,5.992803e+07,6.820723e+05,85399.066100,4.643401e+06,1.373231e+06,38027.858800


In [4]:
class MinMaxScaler:
    def __init__(self, feature_axis=None, minmax_range=(0, 1)):
        """
        Initialize the MinMaxScaler.
        Args:
        feature_axis (int, optional): The axis that represents the feature dimension if applicable.
                                      Use only for 3D data to specify which axis is the feature axis.
                                      Default is None, automatically managed based on data dimensions.
        """
        self.feature_axis = feature_axis
        self.min_ = None
        self.max_ = None
        self.scale_ = None
        self.minmax_range = minmax_range # Default range for scaling (min, max)

    def fit(self, X):
        """
        Fit the scaler to the data based on its dimensionality.
        Args:
        X (np.array): The data to fit the scaler on.
        """
        if X.ndim == 3 and self.feature_axis is not None:  # 3D data
            axis = tuple(i for i in range(X.ndim) if i != self.feature_axis)
            self.min_ = np.min(X, axis=axis)
            self.max_ = np.max(X, axis=axis)
        elif X.ndim == 2:  # 2D data
            self.min_ = np.min(X, axis=0)
            self.max_ = np.max(X, axis=0)
        elif X.ndim == 1:  # 1D data
            self.min_ = np.min(X)
            self.max_ = np.max(X)
        else:
            raise ValueError("Data must be 1D, 2D, or 3D.")

        self.scale_ = self.max_ - self.min_
        return self

    def transform(self, X):
        """
        Transform the data using the fitted scaler.
        Args:
        X (np.array): The data to transform.
        Returns:
        np.array: The scaled data.
        """
        X_scaled = (X - self.min_) / self.scale_
        X_scaled = X_scaled * (self.minmax_range[1] - self.minmax_range[0]) + self.minmax_range[0]
        return X_scaled

    def fit_transform(self, X):
        """
        Fit to data, then transform it.
        Args:
        X (np.array): The data to fit and transform.
        Returns:
        np.array: The scaled data.
        """
        return self.fit(X).transform(X)

    def inverse_transform(self, X_scaled):
        """
        Inverse transform the scaled data to original data.
        Args:
        X_scaled (np.array): The scaled data to inverse transform.
        Returns:
        np.array: The original data scale.
        """
        X = (X_scaled - self.minmax_range[0]) / (self.minmax_range[1] - self.minmax_range[0])
        X = X * self.scale_ + self.min_
        return X

def generate_data(df, sequence_length, n_ahead = 1, colnum = 0):
    #Case without known inputs
    scaler_df = df.copy().shift(n_ahead).rolling(24 * 14).median()
    tmp_df = df.copy() / scaler_df
    tmp_df = tmp_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    scaler_df = scaler_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    def prepare_sequences(df, scaler_df, n_history, n_future):
        X, y, y_scaler = [], [], []
        num_features = df.shape[1]
        
        # Iterate through the DataFrame to create sequences
        for i in range(n_history, len(df) - n_future + 1):
            # Extract the sequence of past observations
            X.append(df.iloc[i - n_history:i].values)
            # Extract the future values of the first column
            y.append(df.iloc[i:i + n_future,colnum:colnum+1].values)
            y_scaler.append(scaler_df.iloc[i:i + n_future,colnum:colnum+1].values)
        
        X, y, y_scaler = np.array(X), np.array(y), np.array(y_scaler)
        return X, y, y_scaler
    
    # Prepare sequences
    X, y, y_scaler = prepare_sequences(tmp_df, scaler_df, sequence_length, n_ahead)
    
    # Split the dataset into training and testing sets
    train_test_separation = int(len(X) * 0.8)
    X_train_unscaled, X_test_unscaled = X[:train_test_separation], X[train_test_separation:]
    y_train_unscaled, y_test_unscaled = y[:train_test_separation], y[train_test_separation:]
    y_scaler_train, y_scaler_test = y_scaler[:train_test_separation], y_scaler[train_test_separation:]
    
    # Generate the data
    X_scaler = MinMaxScaler(feature_axis=2)
    X_train = X_scaler.fit_transform(X_train_unscaled)
    X_test = X_scaler.transform(X_test_unscaled)
    
    y_scaler = MinMaxScaler(feature_axis=2)
    y_train = y_scaler.fit_transform(y_train_unscaled)
    y_test = y_scaler.transform(y_test_unscaled)
    
    y_train = y_train.reshape(y_train.shape[0], -1) 
    y_test = y_test.reshape(y_test.shape[0], -1)
    return X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test



In [6]:
n_aheads = [1, 3, 6, 9, 12, 15][::-1]
models = [
    "GRU",
    "MoE_GRU",
    "KAMoE_GRU",
    "LSTM",
    "MoE_LSTM",
    "KAMoE_LSTM",
 ]

for colnum in range(df.shape[1]):
    results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
    results_rmse = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
    time_results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
    for n_ahead in n_aheads:
        sequence_length = max(45, 5 * n_ahead)
        X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test = generate_data(df, sequence_length, n_ahead, colnum)
        
        for model_id in models:
            
            for run in range(5):
    
                if model_id == 'GRU':
                    model = Sequential([
                        Input(shape=X_train.shape[1:]),
                        GRU(100, return_sequences=True),
                        GRU(100, return_sequences=False),
                        Dense(units=n_ahead, activation='linear')
                    ], name = model_id)
                elif model_id == 'LSTM':
                    model = Sequential([
                        Input(shape=X_train.shape[1:]),
                        LSTM(100, return_sequences=True),
                        LSTM(100, return_sequences=False),
                        Dense(units=n_ahead, activation='linear')
                    ], name = model_id)
                elif model_id == 'MoE_GRU':
                    model = Sequential([
                        Input(shape=X_train.shape[1:]),
                        MoE(GRU(100, return_sequences=True)),
                        GRU(100, return_sequences=False),
                        Dense(units=n_ahead, activation='linear')
                    ], name = model_id)
                elif model_id == 'MoE_LSTM':
                    model = Sequential([
                        Input(shape=X_train.shape[1:]),
                        MoE(LSTM(100, return_sequences=True)),
                        LSTM(100, return_sequences=False),
                        Dense(units=n_ahead, activation='linear')
                    ], name = model_id)
                elif model_id == 'KAMoE_GRU':
                    model = Sequential([
                        Input(shape=X_train.shape[1:]),
                        KAMoE(GRU(100, return_sequences=True)),
                        GRU(100, return_sequences=False),
                        Dense(units=n_ahead, activation='linear')
                    ], name = model_id)
                elif model_id == 'KAMoE_LSTM':
                    model = Sequential([
                        Input(shape=X_train.shape[1:]),
                        KAMoE(LSTM(100, return_sequences=True)),
                        LSTM(100, return_sequences=False),
                        Dense(units=n_ahead, activation='linear')
                    ], name = model_id)
                else:
                    raise ValueError
                
                optimizer = keras.optimizers.Adam(0.001)
                model.compile(optimizer=optimizer, loss='mean_squared_error')
                if run==0:
                    model.summary()
                    
                # Fit the model
                start_time = time.time()
                history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_MAX_EPOCHS, validation_split=0.2, callbacks=callbacks(), shuffle=True, verbose = False)
                end_time = time.time()
                time_results[model_id][n_ahead].append(end_time - start_time)
                # Evaluate the model on the test set
                preds = model.predict(X_test, verbose=False)
                r2 = r2_score(y_true=y_test, y_pred=preds)
                print(end_time - start_time, r2)
                rmse = root_mean_squared_error(y_true=y_test, y_pred=preds)
                results[model_id][n_ahead].append(r2)
                results_rmse[model_id][n_ahead].append(rmse)
        
                del model
                del optimizer
                    
    print('Result', df.columns[colnum])
    print('R2 scores')
    print('Means:')
    display(pd.DataFrame({model_id: {n_ahead: np.mean(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results.keys()}))
    display(pd.DataFrame({model_id: {n_ahead: np.mean(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse.keys()}))
    print('Std:')
    display(pd.DataFrame({model_id: {n_ahead: np.std(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results.keys()}))
    display(pd.DataFrame({model_id: {n_ahead: np.std(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse.keys()}))
    print('Training Times')
    display(pd.DataFrame({model_id: {n_ahead: np.mean(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results.keys()}))
    display(pd.DataFrame({model_id: {n_ahead: np.std(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results.keys()}))

Model: "GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 75, 100)        │        32,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 100)            │        60,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │         1,515 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 94,515 (369.20 KB)

 Trainable params: 94,515 (369.20 KB)

 Non-trainable params: 0 (0.00 B)

...

,GRU,MoE_GRU,KAMoE_GRU,LSTM,MoE_LSTM,KAMoE_LSTM
15,0.003774,0.008013,0.004838,0.004815,0.008522,0.009923
12,0.006119,0.008000,0.004647,0.005373,0.008592,0.008562
9,0.004682,0.005600,0.014858,0.006187,0.003077,0.004673
6,0.004455,0.006175,0.005262,0.002458,0.002350,0.004512
3,0.002285,0.003086,0.002918,0.002211,0.004329,0.006184
1,0.002407,0.002250,0.003668,0.003567,0.005450,0.002041


,GRU,MoE_GRU,KAMoE_GRU,LSTM,MoE_LSTM,KAMoE_LSTM
15,0.000080,0.000169,0.000103,0.000102,0.000177,0.000211
12,0.000129,0.000168,0.000095,0.000115,0.000181,0.000181
9,0.000098,0.000122,0.000317,0.000134,0.000067,0.000101
6,0.000098,0.000136,0.000118,0.000054,0.000051,0.000101
3,0.000052,0.000070,0.000067,0.000048,0.000099,0.000142
1,0.000059,0.000055,0.000089,0.000087,0.000133,0.000050


Training Times


,GRU,MoE_GRU,KAMoE_GRU,LSTM,MoE_LSTM,KAMoE_LSTM
15,31.901321,73.158198,77.604258,24.975881,66.917868,70.821336
12,26.366791,63.252908,64.935228,21.999178,60.302721,62.662763
9,21.056491,54.960584,60.054213,18.366668,50.704089,54.641468
6,22.998008,53.712240,57.872807,19.012076,53.659521,54.614069
3,25.391701,57.433519,62.096873,18.755854,55.645568,56.194430
1,21.666679,59.536334,63.965789,17.386409,51.320873,54.111199


,GRU,MoE_GRU,KAMoE_GRU,LSTM,MoE_LSTM,KAMoE_LSTM
15,1.343977,6.483899,4.190878,1.099729,3.896763,2.451566
12,1.073602,2.350958,2.149985,0.880248,2.755669,1.323170
9,0.333480,3.337489,3.052852,1.535077,2.894741,3.318974
6,1.697174,2.523427,3.113098,1.790458,3.767223,3.220160
3,4.171494,2.115455,3.638489,1.657617,7.209088,3.887616
1,0.608702,5.447388,7.529298,1.315238,1.998203,3.054324
